In [1]:
from gait_analysis import CasiaDataset, settings
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
from torch.utils.data import DataLoader

warnings.filterwarnings('ignore')

# Configure a dataset aggregation of input/labels

We can change the quickly the indexing to access all the angles

In [2]:
from gait_analysis.Config import Config
c = Config()
c.config['indexing']['grouping'] = 'person_sequence_angle'
c.config['pose']['load'] = True
c.config['flow']['load'] = True
c.config['heatmaps']['load'] = True


loading configuration  default


Define the dataset_output 

In [11]:
c.config['dataset_output'] = {
        'data': ["scenes","flows","heatmaps"],
        'label': "annotations"}

Plain and untransformed dataset

In [4]:
dataset_3 = CasiaDataset()

In [5]:
item, labels = dataset_3[1]
scenes = item['scenes']
flows = item['flows']
print(item.keys())

dict_keys(['scenes', 'flows', 'heatmaps'])


# Using the composer to construct reusable transformers

In [6]:
from gait_analysis import Composer

composer = Composer()
transformer = composer.compose()
print(transformer)

In [7]:
print(c.get_indexing_grouping())
test_item, labels = dataset_3[0]

person_sequence_angle


just to test we include add a field annotations 


In [8]:
test_item['annotations'] = labels
transformed_item = transformer(test_item)
print('==> transformed item is: ',test_item.keys())

==> transformed item is:  dict_keys(['scenes', 'flows', 'annotations', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])


In [9]:
item.keys()

dict_keys(['scenes', 'flows', 'heatmaps'])

# Using the transformer and the dataset_output specifications

Printing some samples of the transformed dataset

In [13]:
c.config['dataset_output'] = {
        'data': ["scenes","flows","heatmaps_LAnkle","heatmaps_RAnkle"],
        'label': "annotations"}
transformed_dataset = CasiaDataset(transform=transformer)

for i in range(len(transformed_dataset)):
    print('i: ', i)
    sample, labels = transformed_dataset[i]

    print(i, sample.keys())
    print(labels.size())

    if i == 5:
        break

i:  0
0 dict_keys(['scenes', 'flows', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])
torch.Size([52])
i:  1
1 dict_keys(['scenes', 'flows', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])
torch.Size([52])
i:  2
2 dict_keys(['scenes', 'flows', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])
torch.Size([52])
i:  3
3 dict_keys(['scenes', 'flows', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])
torch.Size([52])
i:  4
4 dict_keys(['scenes', 'flows', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])
torch.Size([52])
i:  5
5 dict_keys(['scenes', 'flows', 'heatmaps_LAnkle', 'heatmaps_RAnkle'])
torch.Size([60])


In [ ]:
sample['annotations']

# Dataloader: using the transformed dataset with the transformer

In [16]:
# dataloader = DataLoader(transformed_dataset, batch_size=4,
#                         shuffle=True, num_workers=4)
dataloader = DataLoader(transformed_dataset, batch_size=5)

In [18]:
for i_batch, sample_batched in enumerate(dataloader):
    # observe 4th batch and stop.
    if i_batch == 3:
        inputs, labels = sample_batched
        print(type(sample_batched))
        break

<class 'list'>


# DEV: The problem is that the dimensions of the label are different for some of the videos. In this case the aggregation works for all the angles in the video but no arbitrary batch_size will work